In [1]:
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline


import pandas as pd
import numpy as np


from bokeh.io import push_notebook, show, output_notebook
from bokeh.charts import Bar, output_file, show
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [59]:
dataSet = pd.read_csv('data/train_clean.csv')
dataSet.head()

,CUST_COD,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,BIRTH_DATE,PAY_DEC,PAY_NOV,PAY_OCT,PAY_SEP,...,BILL_AMT_SEP,BILL_AMT_AUG,BILL_AMT_JUL,PAY_AMT_DEC,PAY_AMT_NOV,PAY_AMT_OCT,PAY_AMT_SEP,PAY_AMT_AUG,PAY_AMT_JUL,DEFAULT PAYMENT JAN
0,53613503,1490.680,M,university,single,NaN,0,0,0,0,...,629.156401,599.283174,608.257067,89.440800,149.06800,68.571280,0.000000,29.81360,59.627200,0
1,35524089,5664.584,M,university,married,04/01/1983,0,0,0,0,...,533.872135,-2.772665,-14.399969,149.068000,159.83071,29.813600,0.000000,0.00000,0.000000,1
2,58777007,6260.856,F,high school,married,02/01/1975,0,0,-1,-1,...,-19.349026,-19.349026,-19.349026,59.627200,532.59015,16.963938,0.000000,0.00000,29.813600,0
3,26913418,8944.080,F,graduate school,married,15/01/1977,-2,-2,-2,-2,...,88.546392,70.330282,253.177091,80.556347,217.22189,88.784901,70.628418,298.46395,216.416922,0
4,41993633,3577.632,F,graduate school,married,09/01/1978,-1,-1,-1,-1,...,0.000000,569.439760,0.000000,119.254400,29.81360,0.000000,569.439760,0.00000,0.000000,1


# CLASS DISTRIBUTION

In [66]:
p = Bar(dataSet,'DEFAULT PAYMENT JAN',color='DEFAULT PAYMENT JAN', title="DATASET", plot_width=400,plot_height=400)
output_notebook()
show(p)

Loading BokehJS ...

# Let's find the dependences between attributes and the result

In [129]:
p0 = Bar(dataSet,'SEX',color='SEX', title="sex distribution", plot_width=400,plot_height=400)
output_notebook()
show(p0)

Loading BokehJS ...

In [74]:
p1 = Bar(dataSet,'DEFAULT PAYMENT JAN',group='SEX',color='SEX', title="grouped by sex", plot_width=400,plot_height=400)
p2 = Bar(dataSet,'DEFAULT PAYMENT JAN',group='MARRIAGE',color='MARRIAGE', title="grouped by marriage", plot_width=400,plot_height=400)
output_notebook()
show(row(p1, p2))

Loading BokehJS ...

In [89]:
import datetime
def get_age(date):
    try:
        return (datetime.datetime.today() - datetime.datetime.strptime(date, '%d/%m/%Y')).days//365
    except Exception as e:
        return date
dataAge=dataSet
dataAge['BIRTH_DATE'] = dataAge['BIRTH_DATE'].apply(get_age)
dataAge.fillna(dataAge.mean())
p3 = Bar(dataAge,'DEFAULT PAYMENT JAN',color='DEFAULT PAYMENT JAN',values='BIRTH_DATE' ,agg='mean', title="age dist", plot_width=400 , plot_height=400)
output_notebook()
show(p3)


Loading BokehJS ...

In [126]:
dataPay=dataSet
col_list= list(dataPay)
col_list=col_list[6:12]
dataPay['payMonths'] = dataPay[col_list].mean(axis=1)
p3 = Bar(dataPay,'DEFAULT PAYMENT JAN',color='DEFAULT PAYMENT JAN',values='payMonths' ,agg='mean', title="grouped by payMonths", plot_width=400 , plot_height=400)
output_notebook()
show(p3)


Loading BokehJS ...